In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
import numpy.testing as npt

In [2]:
scvi_adatas = list(Path("../results/neighbors/").glob("*.h5ad"))

In [5]:
scvi_adatas

[PosixPath('../results/neighbors/Guo_Zhang_2018_NSCLC.qc_apollo-new_4cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Maynard_Bivona_2020_NSCLC.qc_zeus_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Lambrechts_2018_LUAD_6653.qc_apollo-old_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Lambrechts_2018_LUAD_6653.qc_apollo-new_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Zilionis_Klein_2019_NSCLC.qc_apollo-old_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Lambrechts_2018_LUAD_6653.qc_apollo-old_4cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Zilionis_Klein_2019_NSCLC.qc_zeus_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Maynard_Bivona_2020_NSCLC.qc_apollo-new_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Lambrechts_2018_LUAD_6653.qc_apollo-new_4cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Guo_Zhang_2018_NSCLC.qc_apollo-old_8cpus.neighbors.h5ad'),
 PosixPath('../results/neighbors/Maynard_Bivona_2020

In [7]:
def make_df():
    for path in scvi_adatas:
        filename = path.stem
        dataset, rest, _ = filename.split(".")
        _, node, cpus = rest.split("_", maxsplit=3)
        yield dataset, node, cpus, str(path)

samplesheet = pd.DataFrame.from_records(make_df(), columns=["dataset", "node", "cpus", "adata_path"])

In [8]:
adatas = {
    r.Index: sc.read_h5ad(r.adata_path) for r in samplesheet.itertuples()
}

In [10]:
samplesheet.sort_values(["dataset", "node", "cpus"])

,dataset,node,cpus,adata_path
0,Guo_Zhang_2018_NSCLC,apollo-new,4cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_a...
15,Guo_Zhang_2018_NSCLC,apollo-new,8cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_a...
13,Guo_Zhang_2018_NSCLC,apollo-old,4cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_a...
9,Guo_Zhang_2018_NSCLC,apollo-old,8cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_a...
20,Guo_Zhang_2018_NSCLC,zeus,4cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_z...
12,Guo_Zhang_2018_NSCLC,zeus,8cpus,../results/neighbors/Guo_Zhang_2018_NSCLC.qc_z...
8,Lambrechts_2018_LUAD_6653,apollo-new,4cpus,../results/neighbors/Lambrechts_2018_LUAD_6653...
3,Lambrechts_2018_LUAD_6653,apollo-new,8cpus,../results/neighbors/Lambrechts_2018_LUAD_6653...
5,Lambrechts_2018_LUAD_6653,apollo-old,4cpus,../results/neighbors/Lambrechts_2018_LUAD_6653...
2,Lambrechts_2018_LUAD_6653,apollo-old,8cpus,../results/neighbors/Lambrechts_2018_LUAD_6653...


Things are reproducible on the same CPU generation...

In [15]:
npt.assert_almost_equal(adatas[15].obsp["connectivities"].toarray(), adatas[12].obsp["connectivities"].toarray())

... on the same CPU, with different number of cores:

In [16]:
npt.assert_almost_equal(adatas[0].obsp["connectivities"].toarray(), adatas[15].obsp["connectivities"].toarray())

But not with the same number of cores on different CPU generations

In [18]:
npt.assert_almost_equal(adatas[0].obsp["connectivities"].toarray(), adatas[13].obsp["connectivities"].toarray())

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 32972 / 130530625 (0.0253%)
Max absolute difference: 5.364418e-06
Max relative difference: 0.00010679
 x: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],...
 y: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],...

But they are very similar:

In [22]:
npt.assert_almost_equal(adatas[0].obsp["connectivities"].toarray(), adatas[13].obsp["connectivities"].toarray(), decimal=5)

In [23]:
npt.assert_almost_equal(adatas[22].obsp["connectivities"].toarray(), adatas[4].obsp["connectivities"].toarray(), decimal=5)

In [24]:
npt.assert_almost_equal(adatas[7].obsp["connectivities"].toarray(), adatas[18].obsp["connectivities"].toarray(), decimal=5)

In [25]:
npt.assert_almost_equal(adatas[3].obsp["connectivities"].toarray(), adatas[2].obsp["connectivities"].toarray(), decimal=5)